In [1]:
#Solution adapted from Kris C Naik
#GitHub https://github.com/krishnaik06/Stock-Price-Prediction-using-Keras-and-Recurrent-Neural-Networ/blob/master/rnn.py
import matplotlib.pyplot as plt
import pandas as pd
import statistics
import datetime as dt
import urllib.request, json
import pickle
import os
import numpy as np
from pandas import concat

import tensorflow as tf # This code has been tested with TensorFlow 1.6
from sklearn.preprocessing import MinMaxScaler
from pandas.plotting import autocorrelation_plot
#from app.model.preprocessor import Preprocessor as img_prep #image preprocessing

In [2]:
import pandas_datareader as dr
from datetime import datetime
import pandas_datareader.data as web

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv1D, MaxPooling1D

from keras.optimizers import SGD
from keras.utils import np_utils
from scipy import misc
import glob


Using TensorFlow backend.


In [3]:
start=datetime(2015,1,1)
end=datetime(2019,4,1)

In [4]:
# tesla = web.DataReader('TSLA', 'google', start,end)
microsoft = dr.data.get_data_yahoo('MSFT', start=start, end=end)

dataset = microsoft
close_data = microsoft['Close']
dates = pd.DataFrame(close_data.index)

dataset.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2015-01-02,47.419998,46.540001,46.660000,46.759998,27913900.0,42.418739
2015-01-05,46.730000,46.250000,46.369999,46.330002,39673900.0,42.028660
2015-01-06,46.750000,45.540001,46.380001,45.650002,36447900.0,41.411785
2015-01-07,46.459999,45.490002,45.980000,46.230000,29114100.0,41.937946
2015-01-08,47.750000,46.720001,46.750000,47.590000,29645200.0,43.171677


In [5]:
def series_to_supervised(data, n_in=1, n_out=1):
    df = pd.DataFrame(data)
    cols = list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
    # put it all together
    agg = concat(cols, axis=1)
    # drop rows with NaN values
    agg.dropna(inplace=True)
    return agg.values


# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
    return data[:-n_test], data[-n_test:]

In [17]:
n_test = 200  #Test Sample
n_input = 63  #time lag 
n_filter = 8  #filters to use in prediction
n_epochs = 30 #iterations
n_batch = 100 #simulatenous evaluation
n_kernel = 3  #
train, test = train_test_split(dataset['Close'], n_test)


data = series_to_supervised(train, n_in=n_input)
train_x, train_y = data[:, :-1], data[:, -1]
                
train_x = train_x.reshape((train_x.shape[0], train_x.shape[1], 1))

model = Sequential()

model.add(Conv1D(filters=8,kernel_size=n_kernel,kernel_initializer='VarianceScaling', activation='relu', 
                        input_shape=(n_input,1)))
model.add(Conv1D(filters=16,kernel_size=5,strides=1, kernel_initializer='VarianceScaling', activation='relu'))
model.add(MaxPooling1D(pool_size=25))
model.add(Flatten())
model.add(Dense(1))

# model.add(Dense(units=10, kernel_initializer='VarianceScaling',activation='softmax'))
#model.add(Activation('sigmoid'))

model.compile(loss="mse", optimizer='sgd', metrics=['accuracy'])

model.fit(train_x, train_y, batch_size=n_batch, epochs=n_epochs)

Epoch 1/30
806/806 [==============================] - 1s 684us/sample - loss: 82056293721142272.0000 - acc: 0.0000e+00
Epoch 2/30
806/806 [==============================] - 0s 132us/sample - loss: 170429283389938.0312 - acc: 0.0000e+00
Epoch 3/30
806/806 [==============================] - 0s 134us/sample - loss: 118471424484893.2188 - acc: 0.0000e+00
Epoch 4/30
806/806 [==============================] - 0s 174us/sample - loss: 82353685174063.6406 - acc: 0.0000e+00
Epoch 5/30
806/806 [==============================] - 0s 132us/sample - loss: 57246968669440.6328 - acc: 0.0000e+00
Epoch 6/30
806/806 [==============================] - 0s 131us/sample - loss: 39794389945199.1641 - acc: 0.0000e+00
Epoch 7/30
806/806 [==============================] - 0s 130us/sample - loss: 27662482904391.7852 - acc: 0.0000e+00
Epoch 8/30
806/806 [==============================] - 0s 142us/sample - loss: 19229171473761.1914 - acc: 0.0000e+00
Epoch 9/30
806/806 [==============================] - 0s 182us/samp

In [31]:
# Setting Real stock price
real_stock_price = test
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(train)
print(train.head())
history = [x for x in train]

training_set = dataset.iloc[:, 1:2].values
print(training_set.info)

# X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
# predicted_stock_price = regressor.predict(X_test)
x_input = np.array(history[-n_input:]).reshape((1, n_input, 1))
# forecast
predicted_stock_price = model.predict(x_input, verbose=0)

predicted_stock_price = sc.inverse_transform(predicted_stock_price)

# Visualising the results

plt.plot(real_stock_price, color = 'red', label = 'Real Stock Price')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted Stock Price')
plt.title('Stock Price Prediction')
plt.xlabel('Days')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

ValueError: Expected 2D array, got 1D array instead:
array=[ 46.75999832  46.33000183  45.65000153  46.22999954  47.59000015
  47.18999863  46.59999847  46.36000061  45.95999908  45.47999954
  46.24000168  46.38999939  45.91999817  47.13000107  47.18000031
  47.00999832  42.65999985  41.18999863  42.00999832  40.40000153
  41.27999878  41.59999847  41.84000015  42.45000076  42.40999985
  42.36000061  42.59999847  42.38000107  43.09000015  43.86999893
  43.58000183  43.52999878  43.5         43.86000061  44.15000153
  44.09000015  43.99000168  44.06000137  43.84999847  43.88000107
  43.27999878  43.06000137  43.11000061  42.36000061  42.84999847
  42.02999878  41.97999954  41.02000046  41.38000107  41.56000137
  41.70000076  42.5         42.29000092  42.88000107  42.86000061
  42.90000153  41.45999908  41.20999908  40.97000122  40.95999908
  40.65999985  40.72000122  40.29000092  41.54999924  41.52999878
  41.41999817  41.47999954  41.72000122  41.75999832  41.65000153
  42.25999832  42

In [ ]:
from pandas.tseries.offsets import DateOffset

In [ ]:
future_dates = [dataset.index[-1] + DateOffset(days=x) for x in range(1,365)]

In [ ]:
print(regressor.summary())

In [ ]:
x = np.array(future_dates.index)

future_dates['forecast'] = regressor.predict(np.array(future_dates.index))